<a href="https://colab.research.google.com/github/ssharma-ss/ss-tools/blob/main/create_print_CSV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title
# @title: Print converts the files into PRINT CSV for
# Label Matrix and TFormer
# Imports utilities and packages
import warnings
import pandas as pd
from datetime import datetime
import pytz
import os

# Telling function to ignore all warnings
# Remove this
warnings.filterwarnings('ignore')

'''
@function
Remove the spaces in the end of the string.
Note: This does not care if there is spaces
in the middle of the string. There should be no
spaces in the UPC numbers especially.
'''
def strip_end_spaces(val):
  return str(val).strip()

'''
@function
Adds a column called PRINT_QUANTITY
'''
def generate_print_qty(r):
  qty = int(r['QUANTITY'])
  qty = qty + (0.1 * qty)
  qty = round(qty)
  # Edge case of where there are very low volume of print
  # and we forget Feed/Cut
  if qty <= 2:
    return qty * 2
  if qty % 2 == 0:
    return qty
  else:
    return qty + 1

'''
@function
This function converts the file
and spits out a CSV which can be used
in TFormer and Label Matrix both
'''
def print_format(filename):

  # Reading the excel file
  df = pd.read_excel(filename, header=None)

  # Find out where row number where UPCs start
  # based on the header name SORT_CODE.
  slicer = df.index[df[0] == 'SORT_CODE'].tolist()[0]

  # Slice and remove all the rows before this in the dataframe
  df = df.iloc[slicer:]

  # Get a list of all headers
  header_list = df.loc[slicer, :].values.flatten().tolist()

  # Replace all the NaN in the dataframe with empty values
  for series_name, series in df.items():
    df[series_name].fillna('', inplace=True)

  # Rename the dataframe to match the header list
  for i in range(0, len(header_list)):
    header_list[i] = header_list[i].upper()
    df.rename(columns={
        i: header_list[i]
    }, inplace=True)

  # Remove the PRINT_QTY column if it already is present.
  # The idea is that the vendor should not have this column prefilled
  # This should be calculated by the program based on the logic
  # in the function above.
  # This function also removed the index column.
  if 'PRINT_QTY' in header_list:
    df = df.iloc[1:].reset_index().drop(['index', 'PRINT_QTY'], axis=1)
  else:
    df = df.iloc[1:].reset_index().drop(['index'], axis=1)

  # Remove all the rows where there is no UPC_NUMBER or empty rows if any
  df = df.drop(df[df['UPC_NUMBER'] == ''].index)

  # Remove spaces from the ends of all the columns
  df = df.applymap(strip_end_spaces)

  # Calculate the PRINT_QTY column
  df['PRINT_QTY'] = df.apply(generate_print_qty, axis=1)

  # Convert the quantity to INT and then sum them
  df['QUANTITY'] = df['QUANTITY'].astype(int)
  print("Total customer print quantity: ", df['QUANTITY'].sum())
  print("Total actual print quantity: ", df['PRINT_QTY'].sum())

  # Generate a file name for the print
  file_name = ''
  d = datetime.now(pytz.timezone('America/New_York'))
  d = d.strftime('%Y-%m-%d_%H:%M:%S_')
  file_name = "CSV__" + filename + d + "__PRINT.csv"

  # Write all the data to the filename
  with open(file_name, 'w') as csv_file:
      df.to_csv(index=False, path_or_buf=csv_file)

def __main__():
  # Run through the directory of CONTENT on the left hand panel
  directory_path = '/content/'

  # Check for all the files in the directory
  directory_files = os.listdir(directory_path)

  # Get all the files to process
  files_to_process = []

  # Navigate the path and get all the files
  for f in directory_files:

    # Make sure to check if the file has the extension XLSX
    if f[-4:] == "xlsx":
      files_to_process.append(f)

  # Run the print convert for every file in the folder
  for f in files_to_process:
    print_format(f)

  # Outputs the number of files processed
  print(str(len(files_to_process)) + " files processed")

__main__()

Total customer print quantity:  189
Total actual print quantity:  232
1 files processed
